In [1]:
import torch
from torchvision import models
import os

### Inspecting the checkpoint

In [2]:
checkpoint = torch.load('./checkpoint/coffee-diseases-stage2-densenet121-4-ckpt-95.36acc.t7', map_location='cpu')

keys = checkpoint.keys()
print(keys)

print(checkpoint['acc'])
print(checkpoint['epoch'])

keys = checkpoint['model'].keys()

for key in keys:
    print(key)

dict_keys(['model', 'optimizer', 'scaler', 'acc', 'epoch'])
95.36
1
module.features.conv0.weight
module.features.norm0.weight
module.features.norm0.bias
module.features.norm0.running_mean
module.features.norm0.running_var
module.features.norm0.num_batches_tracked
module.features.denseblock1.denselayer1.norm1.weight
module.features.denseblock1.denselayer1.norm1.bias
module.features.denseblock1.denselayer1.norm1.running_mean
module.features.denseblock1.denselayer1.norm1.running_var
module.features.denseblock1.denselayer1.norm1.num_batches_tracked
module.features.denseblock1.denselayer1.conv1.weight
module.features.denseblock1.denselayer1.norm2.weight
module.features.denseblock1.denselayer1.norm2.bias
module.features.denseblock1.denselayer1.norm2.running_mean
module.features.denseblock1.denselayer1.norm2.running_var
module.features.denseblock1.denselayer1.norm2.num_batches_tracked
module.features.denseblock1.denselayer1.conv2.weight
module.features.denseblock1.denselayer2.norm1.weight
mod

### Loading the net

In [3]:
net = models.densenet121(weights='DEFAULT')
net.classifier = torch.nn.Linear(1024, 3)
net

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [4]:
# Load the checkpoint
# checkpoint = torch.load('./checkpoint/coffee-diseases-res50-4-ckpt.t7', map_location='cpu')

# print(checkpoint['model'].items())

In [5]:
# Load the checkpoint
#checkpoint = torch.load('./checkpoint/XXXXX.t7', map_location='cpu')

# Remove the "module" prefix from the keys
new_state_dict = {k.replace('module.', ''): v for k, v in checkpoint['model'].items()}

# Load the modified state_dict into the model
net.load_state_dict(new_state_dict)

<All keys matched successfully>

In [6]:
# print('==> loading from checkpoint..')
# assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
# checkpoint = torch.load('./checkpoint/res50-4-ckpt.t7')

# keys = checkpoint.keys()
# print(keys)

# net.load_state_dict(checkpoint['model'])
# best_acc = checkpoint['acc']
# start_epoch = checkpoint['epoch']

# net = ResNet50()
# checkpoint = torch.load('./checkpoint/coffee-diseases-res50-4-ckpt.t7')
# keys = checkpoint.keys()
# print(keys)
# net.load_state_dict(checkpoint['model'])


# net = checkpoint['net']
# print(net)
# net.load_state_dict(checkpoint['net'])

### Saving the .pth model

In [7]:
torch.save(net.state_dict(), './checkpoint/coffee-diseases-stage2-densenet121-4-ckpt-95.36acc.pth')

### Loading the model

In [8]:
pthPath = f'./checkpoint/coffee-diseases-stage2-densenet121-4-ckpt-95.36acc.pth'
onnxPath = f'./checkpoint/coffee-diseases-stage2-densenet121-4-ckpt-95.36acc.onnx'

# model =  torch.load(pthPath)
# model_dict = model.state_dict()
# net.load_state_dict(model_dict)

### Saving the onnx

In [9]:
!pip install onnx

In [11]:
import onnx
from torch.autograd import Variable

# dummy_input = torch.randn(1, 3, 32, 32)
dummy_input = torch.randn(1, 3, 224, 224)

# Export the model
torch.onnx.export(net,                 # model being run
                  dummy_input,                         # model input (or a tuple for multiple inputs)
                  onnxPath,   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

